In [ ]:
fn = []
save_folder = "MLB"

# Rating
#raw_ratings = open("./" + save_folder + ".lg/import_export/mlb_rosters_2042.txt").readlines()
raw_ratings = open("./" + save_folder + ".lg/import_export/mlb_rosters.txt").readlines()
attributes = [x.strip() for x in [ln for ln in raw_ratings if ln.startswith("//id,")][0].strip()[2:].split(",")]

# print (attributes)

rating_list = [dict(zip(attributes, data.split(","))) for data in raw_ratings if data.startswith("//")== False]
rating_map = dict((p['id'], p) for p in rating_list)

# Games
# OOTP Baseball 23\saved_games\Damanse.lg\news\txt\leagues
for findex in range(1,14188):
    try:
        fn += [x[0:4] + str(findex) + x[4:] for x in open("./" + save_folder +  ".lg/news/txt/leagues/log_{}.txt".format(findex)).readlines()]
    except:
        pass
fn = [ln.strip() for ln in fn]

In [ ]:
singles = 0
reached_third = 0
try_home = 0
scored = 0

import re

def conv(x):
    if type(x) is str:
        x = int(x)
    # 0 -> 20, 200 -> 80
    r = x*60.0/200+20
    return int(round(r/5)*5)

aggregated = {}

for i in range(len(fn)):
    if "SINGLE" in fn[i]:
        #print ("-----")
        #print (fn[i])
        j = i 
        singles += 1
        third_player = -1
        
        flag_reached_third = 0
        flag_try_home = 0
        flag_scored = 0
        
        prev = i
        out = 0
        
        while prev >= 0:
            prev-=1
            if fn[prev].startswith("[%T]"): # start of inning
                break
            if " out" in fn[prev]: # put space to avoid Mike Tr"out"
                if "Batter strikes out." in fn[prev]:
                    # redundant - when run & hit
                    '''
                    [%N]	3-2: Strikes out swinging
                    [%N]	<a href="../players/player_22123.html">Elvis Andrus</a> steals 2nd base
                    [%N]	Batter strikes out.
                    '''
                    continue
                if "reaches first" in fn[prev]:
                    # strike out, not out - 3-2: Strikes out swinging <b>passed ball, reaches first!</b>
                    continue
                #print (fn[prev])
                out += 1
            if "double play" in fn[prev]:
                #print (fn[prev])
                out += 2
            
        while j < len(fn):
            j+=1
            if fn[j].startswith("[%B]"): # next batter
                break
            if fn[j].endswith("</a> to third"): # someone reached third - not just 'to third', to distinguish with 'throw by* * made to thrid'
                match = re.search("player_(\d+).html",fn[j])
                third_player = match.group(1)
                flag_reached_third = 1
            if "Runner from 3rd tries for Home" in fn[j]: #that runner tried home
                p = rating_map[third_player]
                # print (fn[j], conv(p['speed']), conv(p['steal']), conv(p['running']))
                flag_try_home = 1
                if "Runner from 3rd tries for Home, SAFE," in fn[j]:
                    flag_scored = 1
              
        if out >= 3:
            print ("ERROR! " + fn[i])
            break
        
        
        if out != 2:
            # skip two outs
            continue
        
        
        reached_third += flag_reached_third
        try_home += flag_try_home
        scored += flag_scored
        
        if flag_reached_third:
            p = rating_map[third_player]
            spd,stl,run = conv(p['speed']), conv(p['steal']), conv(p['running'])
            for key in [("spd",spd),("stl",stl),("run",run)]:
                s_reached,s_try_home,s_scored = aggregated.get(key, (0,0,0))
                s_reached += 1
                s_try_home += flag_try_home
                s_scored += flag_scored
                aggregated[key] = (s_reached, s_try_home, s_scored)

print (singles)
print (reached_third)
print ("홈 도전/3루 도착 = ",try_home*1.0/reached_third)
print ("득점/홈 도전 = ",scored*1.0/try_home)
         

for tp in ["spd", "stl", "run"]:
    print (tp, "r3 th score th/r3 score/th")
    for rt in range(20,100,5):
        reach,trial,score = aggregated.get((tp, rt), (0,0,0))
        if reach == 0:
            continue
        print (rt, "{:4d}".format(reach), "{:3d}".format(trial), "{:3d}".format(score), "{:5.1f}%".format(trial*100.0/reach), "{:5.1f}%".format(score*100.0/trial))

23555
1884
홈 도전/3루 도착 =  0.6820594479830149
득점/홈 도전 =  0.9299610894941635
spd r3 th score th/r3 score/th
20   13   8   8  61.5% 100.0%
25   68  42  40  61.8%  95.2%
30  196 126 117  64.3%  92.9%
35  293 185 171  63.1%  92.4%
40  501 355 330  70.9%  93.0%
45  278 181 169  65.1%  93.4%
50  119  85  79  71.4%  92.9%
55  180 135 126  75.0%  93.3%
60  105  75  67  71.4%  89.3%
65  100  67  65  67.0%  97.0%
70   16  15  14  93.8%  93.3%
75    7   7   5 100.0%  71.4%
80    8   4   4  50.0% 100.0%
stl r3 th score th/r3 score/th
25   21  16  16  76.2% 100.0%
30   35  23  22  65.7%  95.7%
35  118  76  68  64.4%  89.5%
40  126  90  85  71.4%  94.4%
45  106  66  61  62.3%  92.4%
50   81  54  47  66.7%  87.0%
55  230 148 135  64.3%  91.2%
60  359 241 229  67.1%  95.0%
65  581 415 387  71.4%  93.3%
70  196 138 129  70.4%  93.5%
75   31  18  16  58.1%  88.9%
run r3 th score th/r3 score/th
30   11   5   5  45.5% 100.0%
35   36  19  19  52.8% 100.0%
40  177 122 118  68.9%  96.7%
45  244 147 138  60.2% 